# Load COOP
## Follow these steps to migrate an existing company to Lear

## Setup the environment

In [ ]:
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/colin_api_utils.ipynb
%run /workspaces/lear/tests/data/common/utility.ipynb

import os
import re

## Set the business identifier

In [ ]:
identifier = 'CP0002391'
legal_type = identifier[:re.search(r"\d", identifier).start()]
identifier, legal_type

## Ensure the business does not yet exist!! BAIL IF IT DOES

In [ ]:
if business := legal_api.models.Business.find_by_identifier(identifier):
        print(f'{identifier} already exisit in LEAR, DO NOT ATTENP TO LOAD IT AGAIN')
assert not business, "Business already exists, so we shouldn't try to load creation events."

In [ ]:
echo_colin_api()

In [ ]:
events = get_filing_events_for_corp(legal_type, identifier)
events, len(events)

In [ ]:
filing_event = get_data_load_required_filing_event(legal_type, identifier)
filing_event

In [ ]:
colin_filing_type = filing_event.get('filing_typ_cd')
filing_type = get_filing_type(colin_filing_type)
colin_filing = get_filing(filing_type, legal_type, filing_event)
# colin_filing['filing']['business']['identifier'] = identifier
colin_filing
# filing = create_filing(filing_type, colin_filing, filing_event.get('event_id'), identifier)
# filing.save()
# filing.id

In [ ]:
colin_filing['filing']['business']['identifier'] = identifier
filing = create_filing(filing_type, colin_filing, filing_event.get('event_id'), identifier)
filing.save()
filing.id

### Manually added to the Filer's Queue

eg.
```bash
./q_cli.py -f 57280
```

In [ ]:
print(f'./q_cli.py -f {filing.id}')

In [ ]:
business = legal_api.models.Business.find_by_identifier(identifier)

business.identifier, business.legal_name, business.legal_type

In [ ]:
token = AccountService.get_bearer_token()

### Apply all the filing events

In [ ]:
events

In [ ]:
filing_event = events[6]
filing_event

In [ ]:
colin_filing_type = filing_event.get('filing_typ_cd')
filing_type = get_filing_type(colin_filing_type) # historic
# filing_type = 'historic'
colin_filing = get_filing(filing_type, legal_type, filing_event)
# # colin_filing['filing']['business']['identifier'] = identifier
colin_filing

In [ ]:
colin_filing['filing']['business']['identifier'] = identifier
colin_filing

In [ ]:
r = requests.post(
#     f'{os.getenv("LEGAL_URL")}/{identifier}/filings?draft=true',
    f'{os.getenv("LEGAL_URL")}/{identifier}/filings',
    json=colin_filing,
    headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {token}'}
)

In [ ]:
r.status_code, r.json()

## Affiliate business to business account

### Authenticate

In [ ]:
token = AccountService.get_bearer_token()

In [ ]:
import json

url = "https://auth-api.apps.silver.devops.gov.bc.ca/api/v1/entities"

payload = {
  "businessIdentifier": business.identifier,
  "corpTypeCode": business.legal_type,
  "name": business.legal_name
}

headers = {
  'Authorization': f'Bearer {token}'
}

r = requests.post(url, headers=headers, json=payload)

r.status_code, r.text

### Manual changes
Changed to a paper-only filing after checking on dashboard that the IA does not render yet for COOPs